In [ ]:
device = "cpu"
# device = "cuda"
import time,os,copy,torch,random_equivalent,torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, transforms
from tqdm import tqdm
import pickle
# Pennylane
import pennylane as qml
from pennylane import numpy as np

torch.manual_seed(0)
np.random.seed(0)

# Plotting
import matplotlib.pyplot as plt


# OpenMP: number of parallel threads.
os.environ["OMP_NUM_THREADS"] = "16"


In [ ]:
class_names = [
    "0",
    "1",
    "2",
    "3",
]
num_classes = len(class_names)

import idx2numpy 
import numpy as np 
arr3 = []
arr4 = []
#file1 = './MNIST/TorchvisionDatasetWrapper/raw/train-images-idx3-ubyte'
#file1 = './HybridQRNN/EMNIST/raw/gzip/emnist-digits-train-images-idx3-ubyte' 
file1 = './HybridQRNN/KMNIST/raw/k49-train-imgs.npz' 
#arr = idx2numpy.convert_from_file(file1)
arr = np.load(file1)['arr_0']
#file2 = './MNIST/TorchvisionDatasetWrapper/raw/train-labels-idx1-ubyte'
#file2 = './HybridQRNN/EMNIST/raw/gzip/emnist-digits-train-labels-idx1-ubyte' 
file2 =  './HybridQRNN/KMNIST/raw/k49-train-labels.npz'
arr2 = np.load(file2)['arr_0']

for i in range(len(arr)):
    if arr2[i] in [0,1,2,3]:
        arr3.append(arr[i])
        arr4.append(arr2[i])


In [ ]:
# repeat the images for n_steps
n_steps = 10
# train_sequences = np.tile(train_images[:, None], (1, n_steps, 1, 1))
# test_sequences = np.tile(test_images[:, None], (1, n_steps, 1, 1))
train_images = np.asarray(arr3,dtype=np.float32) / 255.0
train_labels = np.asarray(arr4,dtype=np.int64)

#train_labels -= 6
display(np.shape(train_labels))


In [ ]:
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.axis("off")
    #print(train_labels[i])
    plt.title(class_names[train_labels[i]])

In [ ]:
# test_sequences = train_sequences[:int(len(train_sequences)*0.3)]
# test_labels = train_labels[:int(len(train_labels)*0.3)]
# train_sequences = train_sequences[int(len(train_sequences)*0.7):]
# train_labels = train_labels[int(len(train_labels)*0.7):]
split = np.array_split(train_images,[int(len(train_images)*0.7)])
train_sequences = split[0]
test_sequences = split[1]
print(len(split[0]),len(split[1]))
split = np.array_split(train_labels,[int(len(train_labels)*0.7)])
train_labels = split[0]
test_labels = split[1]
print(len(split[0]),len(split[1]))
datalength = 5000

train_sequences = np.tile(train_sequences[:, None], (1, n_steps, 1, 1))[:datalength]
#train_sequences = np.tile(train_sequences[:, None], (1, n_steps, 1, 1))
test_sequences = np.tile(test_sequences[:, None], (1, n_steps, 1, 1))
train_labels = train_labels[:datalength]
test_labels = test_labels

print(len(test_labels),len(test_sequences),len(train_labels),len(train_sequences))

In [ ]:
def train(input_model, train_x, test_x):
    minibatch_size = 32
    optimizer = torch.optim.Adam(input_model.parameters())
 
    input_model.train()
    for j in range(30):
        train_acc = 0
        loss_acc = 0

        for i in tqdm(range(train_x.shape[0] // minibatch_size)):
            input_model.zero_grad()
 
            batch_in = train_x[i * minibatch_size : (i + 1) * minibatch_size]
            # flatten images
            batch_in = batch_in.reshape((-1,) + train_x.shape[1:-2] + (784,))
            batch_label = train_labels[i * minibatch_size : (i + 1) * minibatch_size]
            output = input_model(torch.tensor(batch_in))
 
            # compute sparse categorical cross entropy loss
            logp = torch.nn.functional.log_softmax(output, dim=-1)
            logpy = torch.gather(logp, 1, torch.tensor(batch_label).view(-1, 1))
            loss = -logpy.mean()
            loss_acc += loss 
 
            loss.backward()
            optimizer.step()
 
            train_acc += torch.mean(
                torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
            )
        train_acc /= i + 1
        #torch.save(model_hybrid,f=f"./saves_class10/{n_qubits}x{q_depth}_{j}_{train_acc.numpy()}.h5")  
        print("Train accuracy (%d): " % j, train_acc.numpy())
        loss_acc /=i + 1
        print("Loss (%d): " % j, loss_acc.detach().numpy())

    input_model.eval()
    test_acc = 0
    for i in range(test_x.shape[0] // minibatch_size):
        batch_in = test_x[i * minibatch_size : (i + 1) * minibatch_size]
        batch_in = batch_in.reshape((-1,) + test_x.shape[1:-2] + (784,))
        batch_label = test_labels[i * minibatch_size : (i + 1) * minibatch_size]
        output = input_model(torch.tensor(batch_in))
 
        test_acc += torch.mean(
            torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
        )
 
    test_acc /= i + 1
 
    print("Test accuracy:", test_acc.numpy())

In [ ]:
# random_model = torch.nn.Sequential(
#     torch.nn.Linear(784, 128),
#     # set spiking_aware_training and a moderate dt
#     random_equivalent.Spike(
#         torch.nn.ReLU(), dt=0.01, spiking_aware_training=True
#     ),
#     random_equivalent.TemporalAvgPool(),
#     torch.nn.Linear(128, 10),
# )

# train(random_model, train_sequences, test_sequences)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, transforms

In [ ]:
# random_model = torch.nn.Sequential(
#     torch.nn.Linear(784, 128),
#     # wrap ReLU in Spike
#     random_equivalent.Spike(torch.nn.ReLU(), spiking_aware_training=True,dt=0.01),
#     # use average pooling layer to average spiking output over time
#     random_equivalent.TemporalAvgPool(),
#     torch.nn.Linear(128, 10),
# )
# display(random_model)
# # train the model, identically to the non-spiking version,
# # except using the time sequences as input
# #train(random_model, train_sequences, test_sequences)

In [ ]:
# class DressedClassicalNet(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.pre_net = nn.Linear(128, 6)
#         self.mid_net = nn.Linear(6,6)
#         self.post_net = nn.Linear(6, 5)

#     def forward(self, input_features):
#         pre_out = self.pre_net(input_features)
#         q_in = torch.tanh(pre_out) * np.pi / 2.0
#         mid_out = self.mid_net(q_in)
#         return self.post_net(mid_out)

In [ ]:
# random_model = torch.load("./HybridQRNN/KMNIST/spikeaware_kmnist.h5")
# random_model = copy.deepcopy(torch.nn.Sequential(*(list(random_model.children())[:-1])))
# for param in random_model.parameters():
#     param.requires_grad = False

# random_model.fc = DressedClassicalNet()
# display(random_model)
# # # Use CUDA or CPU according to the "device" object.
# # random_model = random_model.to(device)


In [ ]:
random_model = torch.nn.Sequential(
    torch.nn.Linear(784, 128),
    # set spiking_aware_training and a moderate dt
    random_equivalent.Spike(
        torch.nn.ReLU(), dt=0.01, spiking_aware_training=True
    ),
    random_equivalent.TemporalAvgPool(),
    torch.nn.Linear(128, 10),
)

In [ ]:
train(random_model, train_sequences, test_sequences)

In [ ]:
torch.save(random_model,"./HybridQRNN/KMNIST/random_kmnist_final.h5")

In [ ]:
class_names = [
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]
num_classes = len(class_names)

import idx2numpy 
import numpy as np 
arr3 = []
arr4 = []
file1 = './train-images-idx3-ubyte' 
arr = idx2numpy.convert_from_file(file1)
file2 = './train-labels-idx1-ubyte' 
arr2 = idx2numpy.convert_from_file(file2)
for i in range(len(arr)):
    if arr2[i] in [6,7,8,9]:
        arr3.append(arr[i])
        arr4.append(arr2[i])


In [ ]:
n_steps = 10
train_images = np.asarray(arr3,dtype=np.float32) / 255.0
train_labels = np.asarray(arr4,dtype=np.int64)
train_labels -= 6
display(train_labels)


In [ ]:
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.axis("off")
    plt.title(class_names[train_labels[i]])

In [ ]:
# test_sequences = train_sequences[:int(len(train_sequences)*0.3)]
# test_labels = train_labels[:int(len(train_labels)*0.3)]
# train_sequences = train_sequences[int(len(train_sequences)*0.7):]
# train_labels = train_labels[int(len(train_labels)*0.7):]
split = np.array_split(train_images,[int(len(train_images)*0.7)])
train_sequences = split[0]
test_sequences = split[1]
print(len(split[0]),len(split[1]))
split = np.array_split(train_labels,[int(len(train_labels)*0.7)])
train_labels = split[0]
test_labels = split[1]
print(len(split[0]),len(split[1]))
datalength = 5000

train_sequences = np.tile(train_sequences[:, None], (1, n_steps, 1, 1))[:datalength]
test_sequences = np.tile(test_sequences[:, None], (1, n_steps, 1, 1))
train_labels = train_labels[:datalength]
test_labels = test_labels

print(len(test_labels),len(test_sequences),len(train_labels),len(train_sequences))


In [ ]:
def test(input_model, test_x):
    minibatch_size = 32
    optimizer = torch.optim.Adam(input_model.parameters())

    input_model.eval()
    test_acc = 0
    for i in range(test_x.shape[0] // minibatch_size):
        batch_in = test_x[i * minibatch_size : (i + 1) * minibatch_size]
        batch_in = batch_in.reshape((-1,) + test_x.shape[1:-2] + (784,))
        batch_label = test_labels[i * minibatch_size : (i + 1) * minibatch_size]
        output = input_model(torch.tensor(batch_in))

        test_acc += torch.mean(
            torch.eq(torch.argmax(output, dim=1), torch.tensor(batch_label)).float()
        )

    test_acc /= i + 1

    print("Test accuracy:", test_acc.numpy())

In [ ]:
model_delta = torch.load("random_model_final_4_class")

In [ ]:
test(model_delta,test_sequences)

In [ ]:
torch.save(random_model,"./MNIST/hybrid_classical_1.h5")

In [ ]:
random_model = torch.load("./hybrid_classical.h5")

In [ ]:
display(random_model)

In [ ]:
n_qubits = 6                # Number of qubits
q_depth = 2                 # Depth of the quantum circuit (number of variational layers)
q_delta = 0.01              # Initial spread of random quantum weights

In [ ]:
def H_layer(nqubits):
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)

def RY_layer(w):
    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)

def entangling_layer(nqubits):
    for i in range(0, nqubits - 1, 2):  
        qml.CNOT(wires=[i, i + 1])
    for i in range(1, nqubits - 1, 2):  
        qml.CNOT(wires=[i, i + 1])

In [ ]:
dev = qml.device("default.qubit", wires=n_qubits)
@qml.qnode(dev, interface="torch")
def quantum_net(q_input_features, q_weights_flat):

    q_weights = q_weights_flat.reshape(q_depth, n_qubits)

    H_layer(n_qubits)

    RY_layer(q_input_features)

    for k in range(q_depth):
        entangling_layer(n_qubits)
        RY_layer(q_weights[k])

    exp_vals = [qml.expval(qml.PauliZ(position)) for position in range(n_qubits)]
    return tuple(exp_vals)
    
class DressedQuantumNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.pre_net = nn.Linear(128, n_qubits)
        self.q_params = nn.Parameter(q_delta * torch.randn(q_depth * n_qubits))
        self.post_net = nn.Linear(n_qubits, 5)

    def forward(self, input_features):
        pre_out = self.pre_net(input_features)
        q_in = torch.tanh(pre_out) * np.pi / 2.0

        q_out = torch.Tensor(0, n_qubits)
        q_out = q_out.to(device)
        for elem in q_in:
            q_out_elem = quantum_net(elem, self.q_params).float().unsqueeze(0)
            q_out = torch.cat((q_out, q_out_elem))

        return self.post_net(q_out)

In [ ]:
model_hybrid = copy.deepcopy(torch.nn.Sequential(*(list(random_model.children())[:-1])))
for param in model_hybrid.parameters():
    param.requires_grad = False

model_hybrid.fc = DressedQuantumNet()
model_hybrid = model_hybrid.to(device)